In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("EmployeeData").getOrCreate()
spark

In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

sales =spark.read.csv('/content/drive/MyDrive/PysparkDemo/retail_sales.csv',header=True,inferSchema=True)


Mounted at /content/drive


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, month, year

#Filter underperforming products
underperforming_df = sales.filter((col("UnitsSold") < 50) | (col("Returns") > 10))
underperforming_df.write.mode("overwrite").csv("output/underperforming_products", header=True)

sales= sales.withColumn("Month", month("Date")).withColumn("Year", year("Date"))

sales = sales.withColumn("Revenue", col("UnitsSold") * col("UnitPrice"))

sales.show()

monthly_revenue = sales.groupBy("StoreID", "Year", "Month").agg(sum("Revenue").alias("MonthlyRevenue"))

#average monthly revenue per store
average_revenue_per_store = monthly_revenue.groupBy("StoreID").agg(avg("MonthlyRevenue").alias("AvgMonthlyRevenue"))

#store-wise average revenue
average_revenue_per_store.write.mode("overwrite").csv("output/average_monthly_revenue", header=True)

average_revenue_per_store.show()


+---------+--------------------+-------+----------+---------+---------+-------+-----+----+-------+
|ProductID|         ProductName|StoreID|      Date|UnitsSold|UnitPrice|Returns|Month|Year|Revenue|
+---------+--------------------+-------+----------+---------+---------+-------+-----+----+-------+
|     1001|     Apple iPhone 14|      1|2025-01-10|      120|    799.0|      4|    1|2025|95880.0|
|     1002|  Samsung Galaxy S23|      1|2025-01-15|       85|    749.0|      6|    1|2025|63665.0|
|     1003|  Dell XPS 13 Laptop|      2|2025-01-18|       30|    999.0|     12|    1|2025|29970.0|
|     1004|   Apple MacBook Air|      2|2025-02-01|       45|   1099.0|      3|    2|2025|49455.0|
|     1005|Sony WH-1000XM5 H...|      3|2025-02-12|       18|    399.0|     14|    2|2025| 7182.0|
|     1006|Bose QuietComfort 45|      3|2025-02-18|       22|    329.0|      2|    2|2025| 7238.0|
|     1007|     Nintendo Switch|      1|2025-03-10|      200|    299.0|      5|    3|2025|59800.0|
|     1008